In [63]:
import os
import requests
import pickle as pkl
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from tqdm import tqdm
from multiprocessing import Pool
import subprocess
from collections import defaultdict

In [67]:
def getCMD(param_path):
    CLASSPATH="/Users/jiaqiangruan/CMU/SearchEngine/out/production/SearchEngine:"+ \
              "/Users/jiaqiangruan/CMU/SearchEngine/lucene-8.1.1/lucene-core-8.1.1.jar:"+ \
              "/Users/jiaqiangruan/CMU/SearchEngine/lucene-8.1.1/QryEvalExtensions.jar:"+ \
              "/Users/jiaqiangruan/CMU/SearchEngine/lucene-8.1.1/lucene-codecs-8.1.1.jar:"+ \
              "/Users/jiaqiangruan/CMU/SearchEngine/lucene-8.1.1/lucene-analyzers-common-8.1.1.jar"
    cmd = "java -classpath %s QryEval %s" % (CLASSPATH, param_path)
    return cmd

In [68]:
def get_trec_eval(output_path):
    userId = 'jruan@andrew.cmu.edu'
    password = 'YOz4vZdm'
    hwId = 'HW5'
    qrels = 'cw09a.adhoc.1-200.qrel.indexed'

    #  Form parameters - these must match form parameters in the web page

    url = 'https://boston.lti.cs.cmu.edu/classes/11-642/HW/HTS/tes.cgi'
    values = { 'hwid' : hwId,				# cgi parameter
               'qrel' : qrels,				# cgi parameter
               'logtype' : 'Detailed',			# cgi parameter "Summary"
               'leaderboard' : 'No'				# cgi parameter
               }

    #  Make the request

    files = {'infile' : (output_path, open(output_path, 'rb')) }	# cgi parameter
    result = requests.post (url, data=values, files=files, auth=(userId, password))

    data = result.text

    data = data[data.index('<pre>'):data.index('</pre>')]
    data = data.split('\n')
    ans = defaultdict(dict)
    for line in data:
        if 'P_10 ' in line:
            ans[line.split()[1]]['P@10'] = float(line.split()[2])
        if 'P_20 ' in line:
            ans[line.split()[1]]['P@20'] = float(line.split()[2])
        if 'P_30 ' in line:
            ans[line.split()[1]]['P@30'] = float(line.split()[2])
        if 'map ' in line:
            ans[line.split()[1]]['MAP'] = float(line.split()[2])
    return ans

def get_ndeval_eval(output_path):
    userId = 'jruan@andrew.cmu.edu'
    password = 'YOz4vZdm'

    #  Form parameters - these must match form parameters in the web page

    url = 'https://boston.lti.cs.cmu.edu/classes/11-642/HW/HTS/nes.cgi'
    values = {'qrel' : 'cw09a.diversity.101-200.qrel.indexed',
              'hwid' : 'HW5'
             }


    #  Make the request

    files = {'infile' : (output_path, open(output_path, 'rb')) }
    result = requests.post (url, data=values, files=files, auth=(userId, password))

    data = result.text
    
    data = data[data.index('<pre>')+len('<pre>'):data.index('</pre>')].strip()
    data = data.split('\n')

    ans = defaultdict(dict)
#     print(list(enumerate(data[0].split(','))))
    for line in data[1:]:
#         print(list(enumerate(line.split(','))))
#         print(len(line.split(',')))
#         break
        tokens = line.split(',')
        if len(tokens) !=23 :continue
        ans[tokens[1]]['P-IA@10'] = float(tokens[18])
        ans[tokens[1]]['P-IA@20'] = float(tokens[19])
        ans[tokens[1]]['aNDCG@20'] = float(tokens[13])

    return ans



In [78]:
for index in ['2.1a','2.1b','2.1c','2.1d','2.1e','2.1f',]:
    print("=========== %s ==============" %index)
#     params_path = "PARAM_DIR/HW5-Exp-%s.param" %index
#     os.system(getCMD(params_path))
    res = get_trec_eval("OUTPUT_DIR/HW5-Exp-%s.teIn"%index)
    for k,v in res.items():
        if k == 'all':
            print(k,v)
    res = get_ndeval_eval("OUTPUT_DIR/HW5-Exp-%s.teIn"%index)
    for k,v in res.items():
        if k == 'amean':
            print(k,v)

=========== 2.1a ==============
all {'MAP': 0.2231, 'P@10': 0.28, 'P@20': 0.345, 'P@30': 0.35}
amean {'P-IA@10': 0.171667, 'P-IA@20': 0.2, 'aNDCG@20': 0.315844}
=========== 2.1b ==============
all {'MAP': 0.1382, 'P@10': 0.34, 'P@20': 0.37, 'P@30': 0.3433}
amean {'P-IA@10': 0.219, 'P-IA@20': 0.234917, 'aNDCG@20': 0.406161}
=========== 2.1c ==============
all {'MAP': 0.1288, 'P@10': 0.28, 'P@20': 0.335, 'P@30': 0.3567}
amean {'P-IA@10': 0.174167, 'P-IA@20': 0.2025, 'aNDCG@20': 0.318724}
=========== 2.1d ==============
all {'MAP': 0.2666, 'P@10': 0.39, 'P@20': 0.385, 'P@30': 0.3933}
amean {'P-IA@10': 0.239167, 'P-IA@20': 0.225417, 'aNDCG@20': 0.41438}
=========== 2.1e ==============
all {'MAP': 0.1706, 'P@10': 0.4, 'P@20': 0.425, 'P@30': 0.41}
amean {'P-IA@10': 0.2805, 'P-IA@20': 0.301, 'aNDCG@20': 0.490166}
=========== 2.1f ==============
all {'MAP': 0.1828, 'P@10': 0.42, 'P@20': 0.455, 'P@30': 0.41}
amean {'P-IA@10': 0.293, 'P-IA@20': 0.3165, 'aNDCG@20': 0.547628}


In [82]:
for index in ['3.1a', '3.1b', '3.1c', '3.1d',
              '3.2a', '3.2b', '3.2c', '3.2d',
              '3.3a', '3.3b', '3.3c', '3.3d',
              '3.4a', '3.4b', '3.4c', '3.4d',]:
    print("=========== %s ==============" %index)
#     params_path = "PARAM_DIR/HW5-Exp-%s.param" %index
#     os.system(getCMD(params_path))
    res = get_ndeval_eval("OUTPUT_DIR/HW5-Exp-%s.teIn"%index)
    for k,v in res.items():
        if k == 'amean':
            print("%.5f" % v['P-IA@10'])
            print("%.5f" % v['P-IA@20'])
            print("%.5f" % v['aNDCG@20'])
#             print(k,v)
    res = get_trec_eval("OUTPUT_DIR/HW5-Exp-%s.teIn"%index)
    for k,v in res.items():
        if k == 'all':
            print("%.4f" % v['P@10'])
            print("%.4f" % v['P@20'])
            print("%.4f" % v['MAP'])
#             print(k,v)


=========== 3.1a ==============
0.21683
0.23842
0.36887
0.3000
0.3550
0.1407
=========== 3.1b ==============
0.21933
0.24158
0.41902
0.3400
0.3600
0.1422
=========== 3.1c ==============
0.21467
0.22825
0.40411
0.3400
0.3600
0.1342
=========== 3.1d ==============
0.19550
0.20025
0.40309
0.3400
0.3500
0.1279
=========== 3.2a ==============
0.17167
0.20000
0.31584
0.2800
0.3450
0.1213
=========== 3.2b ==============
0.16667
0.20500
0.31631
0.2700
0.3450
0.1247
=========== 3.2c ==============
0.18167
0.20875
0.32084
0.2900
0.3350
0.1323
=========== 3.2d ==============
0.22183
0.23108
0.40655
0.3500
0.3550
0.1426
=========== 3.3a ==============
0.30533
0.30225
0.48695
0.4600
0.4250
0.1913
=========== 3.3b ==============
0.27967
0.30725
0.51528
0.4100
0.4350
0.1747
=========== 3.3c ==============
0.28633
0.29808
0.53560
0.4100
0.4300
0.1763
=========== 3.3d ==============
0.19722
0.19514
0.61049
0.3000
0.3500
0.2800
=========== 3.4a ==============
0.23917
0.22542
0.41438
0.3900
0.3850
0.1787

In [84]:
for index in ['4.1a', '4.1b', '4.1c', '4.1d',
              '4.2a', '4.2b', '4.2c', '4.2d',
              '4.3a', '4.3b', '4.3c', '4.3d',]:
    print("=========== %s ==============" %index)
    params_path = "PARAM_DIR/HW5-Exp-%s.param" %index
    os.system(getCMD(params_path))
    res = get_ndeval_eval("OUTPUT_DIR/HW5-Exp-%s.teIn"%index)
    for k,v in res.items():
        if k == 'amean':
            print("%.5f" % v['P-IA@10'])
            print("%.5f" % v['P-IA@20'])
            print("%.5f" % v['aNDCG@20'])
#             print(k,v)
    res = get_trec_eval("OUTPUT_DIR/HW5-Exp-%s.teIn"%index)
    for k,v in res.items():
        if k == 'all':
            print("%.4f" % v['P@10'])
            print("%.4f" % v['P@20'])
            print("%.4f" % v['MAP'])
#             print(k,v)

=========== 4.1a ==============
0.20917
0.20750
0.35386
0.3300
0.3400
0.0724
=========== 4.1b ==============
0.20500
0.21875
0.35210
0.3500
0.3400
0.0685
=========== 4.1c ==============
0.21900
0.23492
0.40616
0.3400
0.3700
0.1382
=========== 4.1d ==============
0.21967
0.25317
0.46936
0.3300
0.3550
0.2475
=========== 4.2a ==============
0.25583
0.23333
0.48004
0.3700
0.3800
0.1163
=========== 4.2b ==============
0.30517
0.28258
0.50670
0.4500
0.4250
0.1185
=========== 4.2c ==============
0.28050
0.30100
0.49017
0.4000
0.4250
0.1706
=========== 4.2d ==============
0.27283
0.27992
0.53431
0.4100
0.3900
0.2812
=========== 4.3a ==============
0.26083
0.23833
0.49658
0.3900
0.3800
0.1508
=========== 4.3b ==============
0.27917
0.29925
0.49863
0.4300
0.4550
0.1228
=========== 4.3c ==============
0.29300
0.31650
0.54763
0.4200
0.4550
0.1828
=========== 4.3d ==============
0.29700
0.29367
0.51267
0.4600
0.4450
0.3069
